[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jlfvindel/SW-KG/blob/main/Grafos_RDF/Wikidata-CONSTRUCT.ipynb)

## Resumen de este cuaderno
+ Se remite una consulta a Wikidata de tipo CONSTRUCT, de la que se espera un grafo como respuesta
+ Se reciben los resultados, se presentan en formato Turtle y XML y se descargan dos ficheros en esos respectivos formatos
+ Se construye otro grafo distinto a partir del primero mediante una consulta CONSTRUCT local

In [ ]:
# Comente o descomente conforme los paquetes estén instalados
!pip install sparqlwrapper rdflib 

## Diseño de la consulta CONSTRUCT

### Primero, como consulta SELECT
Aprovechando el interfaz web de consulta de Wikidata, se experimenta con una consulta SELECT previa, con las siguientes características:
+ se acotan las entidades de interés a **los museos localizados en España**
+ de estos ítem ?museo nos interesan todas las relaciones (museo, relacionado_por_X_con, objeto) almacenadas en Wikidata como enunciados directos; para todo tipo de relación posible del museo con todo tipo de objeto
+ adicionalmente se solicita el tipo de la relación, es decir, qué tipo sintáctico se espera como objeto, así como la clase semántica a la que pertenece el objeto

> **Esta consulta se puede ver y ejecutar externamente en** [este enlace](https://query.wikidata.org/#%23%201.%20Items%20%28%3Fmuseo%29%2C%20instancias%20de%20cualquier%20subclase%20de%20museo%20%28Q33506%29%20en%20pa%C3%ADs%20España%20%28Q29%29.%0A%23%20%20%20%20Se%20busca%20toda%20tripleta%20con%20%3Fmuseo%20como%20sujeto%3A%20%3Fmuseo%20%3FpD%20%3Fo%20.%0A%23%202.%20En%20realidad%20en%201%20se%20buscan%20tan%20sólo%20las%20tripletas%20que%20sean%20%27enunciados%20directos%27%3A%20%3Fs%20%3FpD%20%3Fo%2C%20as%C3%AD%20que%0A%23%20%20%20%20%3FpD%20debe%20cumplir%20una%20restricción.%20Si%20%3Fp%20wikibase%3AdirectClaim%20%3FpD%2C%20entonces%20%3FpD%20es%20la%20variante%20sintáctica%0A%23%20%20%20%20adecuada%20para%20ser%20usada%20en%20enunciados%20directos.%0A%23%203.%20Los%20objetos%20%3Fo%2C%20en%20caso%20de%20no%20ser%20literales%2C%20pueden%20pertenecer%20a%20alguna%20clase%0A%0ASELECT%20%20%3FmuseoLabel%20%3FpLabel%20%3FoLabel%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%3Fmuseo%20%3FpD%20%3Fo%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%3FpTipo%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%3FoClase%20%3FoClaseLabel%20%20%20%20%20%20%20%0AWHERE%20%7B%0A%20%20%3Fp%20rdf%3Atype%20wikibase%3AProperty%20%3B%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%23%202%0A%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3AdirectClaim%20%3FpD%20%3B%20%0A%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3ApropertyType%20%3FpTipo.%20%0A%20%20%0A%20%20%3Fmuseo%20wdt%3AP31%2Fwdt%3AP279%2a%20wd%3AQ33506%20%3B%20%20%20%20%20%20%20%20%20%20%20%20%23%201%0A%20%20%20%20%20%20%20%20%20%20wdt%3AP17%20wd%3AQ29%20%3B%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%3FpD%20%3Fo%20.%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%0A%20%20OPTIONAL%20%7B%3Fo%20wdt%3AP31%20%3FoClase%7D%20.%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%23%203%0A%20%20%0A%20%20%23%20De%20cualquier%20%3Fitem%2C%20este%20servicio%20proporciona%20su%20etiqueta%20en%20%3FitemLabel%20%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22es%22%2C%22ca%22%2C%22eu%22%2C%22gl%22%2C%22en%22%2C%22fr%22.%20%7D%0A%7D%0AORDER%20BY%20%3Fmuseo%0ALIMIT%20200000). La consulta devuelve una colección de n-tuplas, 9-tuplas en este caso, con tantos componente como variables solicitadas como respuesta en SELECT.

### Diseño de la consulta CONSTRUCT
También esta consulta CONSTRUCT se puede ejecutar y refinar directamente sobre el interfaz web de Wikidata. El patrón WHERE de esta consulta es similar al anterior y devuelve una n-tupla resultante cada vez que una instanciación de todas las variables de ese patrón coincidan localmente con el grafo consultado (el de todo Wikidata).
> [Este enlace](https://query.wikidata.org/#CONSTRUCT%20%7B%0A%20%20%23%20Por%20el%20cuerpo%20del%20WHERE%2C%20los%20%3Fsujeto%20son%20museos%20en%20España.%0A%20%20%3Fsujeto%20%3FpropD%20%3Fobjeto%20%3B%0A%20%20%20%20%20%20%20%20%20%20rdfs%3Alabel%20%3FsujetoLabel%20%3B%0A%20%20%20%20%20%20%20%20%20%20rdf%3Atype%20%3FsujetoClase%20.%0A%20%20%20%0A%20%20%3Fprop%20rdf%3Atype%20wikibase%3AProperty%20%3B%0A%20%20%20%20%20%20%20%20rdfs%3Alabel%20%3FpropLabel%20%3B%0A%20%20%20%20%20%20%20%20wikibase%3AdirectClaim%20%3FpropD%20%3B%0A%20%20%20%20%20%20%20%20wikibase%3ApropertyType%20%3FpropTipoObjeto%20.%0A%20%20%0A%20%20%3Fobjeto%20rdfs%3Alabel%20%3FobjetoLabel%20%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20wdt%3AP31%20%3FobjetoClase%20.%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%7D%0AWHERE%20%7B%0A%20%20%3Fprop%20rdf%3Atype%20wikibase%3AProperty%20%3B%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3AdirectClaim%20%3FpropD%20%3B%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20wikibase%3ApropertyType%20%3FpropTipoObjeto.%20%0A%20%20%0A%20%20%3Fsujeto%20wdt%3AP31%2Fwdt%3AP279%2a%20wd%3AQ33506%20%3B%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20wdt%3AP31%20%3FsujetoClase%20%3B%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20wdt%3AP17%20wd%3AQ29%20%3B%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%3FpropD%20%3Fobjeto%20.%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%0A%20%20OPTIONAL%20%7B%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20%3Fobjeto%20wdt%3AP31%20%3FobjetoClase%20.%0A%20%20%7D%20.%20%20%20%20%20%20%20%20%20%20%0A%20%20%0A%20%20%23%20De%20cualquier%20%3Fitem%2C%20este%20servicio%20proporciona%20su%20etiqueta%20en%20%3FitemLabel%20%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22es%22%2C%22ca%22%2C%22eu%22%2C%22gl%22%2C%22en%22%2C%22fr%22.%20%7D%0A%7D) **muestra la consulta CONSTRUCT, que puede ejecutarse externamente**. 

En la consulta SELECT anterior, cada una de esas n-tuplas resultantes se mostraban como una fila de resultados de la tabla. Las consultas CONSTRUCT **siempre devuelven tripletas (sujeto, propiedad, objeto), es decir, un grafo resultante**. Las n-tuplas de instancias que va facilitando el cuerpo del WHERE se usan para configurar el grafo tal y como se declara en el cuerpo de CONSTRUCT: con los espacios de nombre que se desee y con la relación entre entidades que se quiera. En este ejemplo, salvo la inserción de alguna tripleta nueva, se mantiene en el grafo resultante la relación que tenían las entidades en Wikidata.

In [1]:
consulta = '''
CONSTRUCT {
  # Por el cuerpo del WHERE, los ?sujeto son museos en España.
  ?sujeto ?propD ?objeto ;
          rdfs:label ?sujetoLabel ;
          rdf:type ?sujetoClase .
   
  ?prop rdf:type wikibase:Property ;
        rdfs:label ?propLabel ;
        wikibase:directClaim ?propD ;
        wikibase:propertyType ?propTipoObjeto .
  
  ?objeto rdfs:label ?objetoLabel ;
              wdt:P31 ?objetoClase .              
}
WHERE {
  ?prop rdf:type wikibase:Property ;                 
            wikibase:directClaim ?propD ;             
            wikibase:propertyType ?propTipoObjeto. 
  
  ?sujeto wdt:P31/wdt:P279* wd:Q33506 ;              
          wdt:P31 ?sujetoClase ;                     
          wdt:P17 wd:Q29 ;                           
          ?propD ?objeto .                           
  
  OPTIONAL {                                         
    ?objeto wdt:P31 ?objetoClase .
  } .          
  
  # De cualquier ?item, este servicio proporciona su etiqueta en ?itemLabel 
  SERVICE wikibase:label { bd:serviceParam wikibase:language "es","ca","eu","gl","en","fr". }
}
'''

## Ejecución, recepción y presentación

In [2]:
from SPARQLWrapper import SPARQLWrapper, RDFXML, TURTLE
from rdflib import Graph, URIRef, Literal, RDF, RDFS

### Ejecución y recepción de resultados
Se remite la consulta a Wikidata. Puede tardar algo de tiempo e incluso interrumpirse en algún caso.

In [3]:
servidor = "https://query.wikidata.org/sparql" 
cliente = SPARQLWrapper(servidor)
# Se fija el retorno a RDFXML porque se espera un grafo
cliente.setReturnFormat(RDFXML)

cliente.setQuery(consulta)
# Ejecución de la consulta y recepción en un objeto QueryResult
resp_obj = cliente.query()

In [4]:
# Conversión a un grafo RDF internamente vía RDFLib
resp_grafo = resp_obj.convert()

print(f'El grafo RDFLib resultante consta de {len(resp_grafo)} tripletas')

El grafo RDFLib resultante consta de 15297 tripletas


### Secuenciación Turtle del grafo
El grafo RDFLib `resp_grafo` se puede secuenciar textualmente en varios formatos para ser impreso o almacenado en ficheros.

In [ ]:
resp_f_turtle = resp_grafo.serialize(format="turtle")
print(resp_f_turtle[:5000])

In [ ]:
resp_f_xml = resp_grafo.serialize(format="xml")
print(resp_f_xml[:5000])

## Descarga del grafo como fichero

### Descarga desde Google Colab
Si este cuaderno se está ejecutando desde Google colab:

In [ ]:
from google.colab import files

with open('resp_f_xml.xml', 'w') as f:
  f.write(resp_f_xml)
files.download('resp_f_xml.xml')

In [ ]:
with open('resp_f_turtle.ttl', 'w') as f:
  f.write(resp_f_turtle)
files.download('resp_f_turtle.ttl')

### Descarga desde ejecución local
Descomente y reemplace, en caso de ejecución local:

In [ ]:
#fichero_turtle = 'reemplace esto con el camino y el nombre del fichero .ttl'
#resp_grafo.serialize(destination=fichero_turtle, format='turtle')

In [ ]:
#fichero_xml = 'reemplace esto con el camino y el nombre del fichero .xml'
#resp_grafo.serialize(destination=fichero_xml, format='xml')

## Consultas CONSTRUCT sobre el grafo local
Un grafo RDFLib local puede ser consultado por dos métodos: SPARQL o mediante métodos rdflib propios. La consulta SPARQL puede ser de tipo SELECT o CONSTRUCT. En este último caso, el resultado es un nuevo grafo local RDFLib.

### Consulta
En WHERE: se localizan todas las propiedades ?pD con las que se van a construir enunciados directos (?museo ?pD ?o), donde ?museo se ha restringido a la entidad que tiene por etiqueta en español "Museo del Prado". En CONSTRUCT se replican estos resultados: todos los enunciados directos que tienen por sujeto a Museo_del_Prado, así como las descripciones de las propiedades y objetos que intervienen en esos enunciados.

Esta consulta CONSTRUCT puede demorarse bastante.

In [ ]:
q = '''
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX wd: <http://www.wikidata.org/entity/>
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
PREFIX wikibase: <http://wikiba.se/ontology#>
CONSTRUCT {
    ?museo ?pD ?o ;
           rdfs:label "Museo del Prado"@es .
    ?prop rdf:type rdf:Property ;
          wikibase:directClaim ?pD ;
          rdfs:label ?propEtiq .
    ?o rdfs:label ?oEtiq .
}
WHERE {
   ?prop rdf:type wikibase:Property ;
         wikibase:directClaim ?pD;
         rdfs:label ?propEtiq .
   ?museo rdfs:label "Museo del Prado"@es ;
          ?pD ?o .
   ?o rdfs:label ?oEtiq .
}
'''

g_local1 = resp_grafo.query(q).graph

### Resultados en formato Turtle

In [ ]:
g_local1_ttl = g_local1.serialize(format="turtle")
print(g_local1_ttl)

### Visualización de resultados

Se añade una función de visualización, via networkx y graphviz, que se encuentra en el siguiente cuaderno Colab: https://colab.research.google.com/github/joerg84/Graph_Powered_ML_Workshop/blob/master/Sparql.ipynb.

In [ ]:
import networkx as nx
import io
import pydotplus
from IPython.display import display, Image
from rdflib.tools.rdf2dot import rdf2dot

In [ ]:
def visualiza(g):
    stream = io.StringIO()
    rdf2dot(g, stream, opts = {display})
    dg = pydotplus.graph_from_dot_data(stream.getvalue())
    png = dg.create_png()
    display(Image(png))

In [ ]:
visualiza(g_local1)

## Consulta similar a CONSTRUCT mediante métodos rdflib

(1) Se busca la entidad que tiene por etiqueta "Museo del Prado" en español

In [9]:
etiq, idioma = "Museo del Prado", "es"
sujeto_fijo = resp_grafo.value(None, RDFS.label, Literal(etiq,lang=idioma))
print(sujeto_fijo)

http://www.wikidata.org/entity/Q160112


(2) Se genera un nuevo grafo y se añaden todos los enunciados directos (s,p,o) de los que es sujeto esa entidad Museo_del_Prado

In [10]:
g_local2 = Graph()
# Todos los enunciados directos (s,p,o) de los que es sujeto la entidad Museo_del_Prado
for prop, direct, propD in resp_grafo.triples( (None,URIRef('http://wikiba.se/ontology#directClaim'),None) ):
    for s,p,o in resp_grafo.triples( (sujeto_fijo,propD,None) ):
        g_local2.add( (s,p,o) )

g_local2_ttl = g_local2.serialize(format="turtle")
print(g_local2_ttl)

@prefix ns1: <http://www.wikidata.org/prop/direct/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.wikidata.org/entity/Q160112> ns1:P1015 "90153245"^^xsd:string ;
    ns1:P1017 "ADV10097072"^^xsd:string,
        "ADV12006249"^^xsd:string,
        "ADV12006253"^^xsd:string ;
    ns1:P1037 <http://www.wikidata.org/entity/Q28992858> ;
    ns1:P112 <http://www.wikidata.org/entity/Q36234> ;
    ns1:P1174 1175296.0,
        3497345.0 ;
    ns1:P1207 "n97045519"^^xsd:string ;
    ns1:P1273 "a10128128"^^xsd:string ;
    ns1:P1296 "0052537"^^xsd:string ;
    ns1:P131 <http://www.wikidata.org/entity/Q2807> ;
    ns1:P1329 "+34 913 30 28 00"^^xsd:string ;
    ns1:P1368 "000093261"^^xsd:string ;
    ns1:P1375 "000198133"^^xsd:string ;
    ns1:P1417 "topic/Prado-Museum"^^xsd:string ;
    ns1:P1424 <http://www.wikidata.org/entity/Q26139861> ;
    ns1:P1435 <http://www.wikidata.org/entity/Q23712> ;
    ns1:P1436 4.0,
        155.0,
        932.0,
        981.0,
        7825.0,
    